In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve, IsotonicRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import brier_score_loss, log_loss
from sklearn.model_selection import KFold



def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

# We start with the initial probabilities from the uncalibrated model
y_probs = softmax(all_train_logits_df.to_numpy())

#suitable for multiclass here
lb = LabelBinarizer()
y_true_binarized = lb.fit_transform(y_train_df)
num_classes = y_probs.shape[1]

# #Using cross validation is the best practice

n_splits = 5 # Using 5 splits, this can be changed. It has to be similar to the one used for temperature scaling to maintain fairness.
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

final_calibrated_probs_iso = np.zeros_like(y_probs)

print(f"Starting {n_splits}-fold cross-validation to generate Isotonic Regression probabilities...")

for fold, (train_index, val_index) in enumerate(kf.split(y_probs)):
    print(f"--- Processing Fold {fold+1}/{n_splits} ---")

    ## Split the initial probabilities and true labels into training and validation sets for this fold
    y_probs_train, y_probs_val = y_probs[train_index], y_probs[val_index]
    y_true_train, y_true_val = y_true_binarized[train_index], y_true_binarized[val_index]

    ## This will hold the calibrated probabilities for the validation set of THIS FOLD
    calibrated_probs_val = np.zeros_like(y_probs_val)

    ## Train a separate Isotonic Regression model for each class on the TRAIN portion of the fold
    for i in range(num_classes):
        ir = IsotonicRegression(out_of_bounds='clip')


        ir.fit(y_probs_train[:, i], y_true_train[:, i])


        calibrated_probs_val[:, i] = ir.transform(y_probs_val[:, i])

     ##NOTE: Renormalize them before using-using your method (not shown here)
    final_calibrated_probs_iso[val_index] = calibrated_probs_val

print("\nCross-validation complete. Final calibrated probabilities have been generated.")





def stratified_brier_score_per_class(y_true_binarized, y_prob):
    brier_scores = []
    y_true_flat = y_true_binarized.argmax(axis=1) # Convert from one-hot to class indices
    for i in range(y_prob.shape[1]):
        pos_indices = (y_true_flat == i)
        neg_indices = (y_true_flat != i)

        N_pos = np.sum(pos_indices)
        N_neg = np.sum(neg_indices)

        if N_pos > 0:
            brier_score_pos = np.sum((1 - y_prob[pos_indices, i])**2) / N_pos
        else:
            brier_score_pos = 0

        if N_neg > 0:

            brier_score_neg = np.sum((y_prob[neg_indices, i])**2) / N_neg
        else:
            brier_score_neg = 0

        stratified_brier = (brier_score_pos + brier_score_neg) / 2
        brier_scores.append(stratified_brier)
    return brier_scores


print('\n Final metrics for Isotonic Regression (from cross-validated probabilities)')






## Compute final metrics using the fully populated `final_calibrated_probs_iso`

final_classwise_log_losses = [log_loss(y_true_binarized[:, i], final_calibrated_probs_iso[:, i]) for i in range(num_classes)]

## Print class-wise results
for i in range(num_classes):
    print(f'Class {i}:')
    print(f'    Log-loss: {final_classwise_log_losses[i]:.6f}')
    print(f'    Stratified Brier score: {final_stratified_brier_scores[i]:.6f}')

# Print overall results, display the mean of log losses if needed.

print(f'Overall mean Stratified Brier score: {np.mean(final_stratified_brier_scores):.6f}')



def plot_calibration_curve(y_true_binarized, probs, title):
    plt.figure(figsize=(13, 8))
    y_true_flat = y_true_binarized.argmax(axis=1)
    for i in range(probs.shape[1]):
        prob_true, prob_pred = calibration_curve(y_true_flat == i, probs[:, i], n_bins=10)
        plt.plot(prob_pred, prob_true, marker='o', label=f'Class {i}')

    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('Mean predicted probability', fontweight='bold', fontsize=24)
    plt.ylabel('Fraction of positives', fontweight='bold', fontsize=24)
    plt.title(title, fontweight='bold', fontsize=24)
    plt.xticks(fontsize=24, fontweight='bold')
    plt.yticks(fontsize=24, fontweight='bold')
    plt.legend(fontsize=15)
    plt.show()

##Call this function to plot the calibration plot
plot_calibration_curve(y_true_binarized, final_calibrated_probs_iso, 'Calibration Plot - Isotonic Regression')






